# Assignment 3 - Comic Book using python


We'll be making customize function where we will be adding the text to images first, then after adding text to images border will be added.
After getting the bordered images we'll be horizontally stacking up the images in random number per line each.
Once Images are horizontally stacked, we'll stack them vertically.
And finally we'll convert them to pdf

In [ ]:
# To add text into image we have created a function naming addText which will add text to the image

from PIL import Image as pili, ImageDraw as pild, ImageFont as pilf

def addText(directory , img ,caption ,filename):
    
    
    TINT_COLOR = (0, 0, 0)  # Black
    TRANSPARENCY = .25  # Degree of transparency, 0-100%
    OPACITY = int(255 * TRANSPARENCY)

    from PIL import Image, ImageFont, ImageDraw

    img = pili.open(img).convert('RGBA')

    overlay = pili.new('RGBA', img.size, TINT_COLOR+(0,))
    draw = pild.Draw(overlay)
    font = pilf.truetype("arial.ttf", 40)
    #x, y = (img.width - 510, img.height-100)
    text = caption
    w, h = font.getsize(text)
    num_lines = len(text.split('\n'))
    print(num_lines, w, h)
    x, y = 40, img.height - ((num_lines-0.1*num_lines)*h + 50)
    #draw.rectangle((x, y, x + w, y + h), fill='black')
    #draw.rectangle((x, y, x + w, y + 4*h), fill=TINT_COLOR+(OPACITY,))
    draw.rectangle((x, y, x + img.width - 200, y + (num_lines-0.1*num_lines)*h), fill=TINT_COLOR+(OPACITY,))
    draw.text((x, y), text, fill=(209, 239, 8), font=font)

    # Alpha composite these two images together to obtain the desired result.
    img = pili.alpha_composite(img, overlay)
    img = img.convert("RGB") # Remove alpha for saving in jpg format.

    img.save(directory+filename)

In [ ]:
# To read the \n from dataframe we manipulated using replace function

import pandas as pd

# Reading the data from "English" column
df = pd.read_excel("C:/Users/divya/Documents/Cartoons/dialogue.xlsx")
df['English']=df['English'].replace(r'\\n', r'\n', regex=True)

# Reading the data from "Hindi" column
df['Hindi']=df['Hindi'].replace(r'\\n', r'\n', regex=True)
df['Hindi']

In [ ]:
# To get the image name in ascending order so that we can zip the dialogue with the image  we used custom function

import re

def tryint(s):
    try:
        return int(s)
    except:
        return s

def alphanum_key(s):
    """ Turn a string into a list of string and number chunks.
        "z23a" -> ["z", 23, "a"]
    """
    return [ tryint(c) for c in re.split('([0-9]+)', s) ]

def sort_nicely(l):
    """ Sort the given list in the way that humans expect.
    """
    l.sort(key=alphanum_key)
    return l

In [ ]:
# Get the PNG files present in that location

import os

d_directory = r"D:/subtitles/"

all_files=[]

for filename in os.listdir(d_directory):
    if filename.endswith(".PNG") or filename.endswith(".png"):
#         print(filename)
        all_files.append(filename)

    else:
        print("Caption not added "+filename)

# sorting the filename so that it is collected in ascending order
all_files=sort_nicely(all_files)
all_files

In [ ]:
# hindiText Function is called up whenever Hindi text needs to be added in image

def hindiText(directory, image,caption , filename):
    
    from PIL import Image as pili, ImageDraw as pild, ImageFont as pilf 
    from PIL import Image, ImageDraw, ImageFont, ImageFilter
    #configuration
    img = pili.open(image)
    
    font_size=36
    width=500
    height=100
    back_ground_color=(255,255,255)
    font_size=36
    font_color=(209, 239, 8)
    unicode_text = caption

#     im  =  Image.new ( "RGB", (width,height), back_ground_color )
    draw  =  ImageDraw.Draw ( img)
    #unicode_font = ImageFont.truetype("DENGL.ttf", font_size)
    unicode_font = ImageFont.truetype("c:/windows/fonts/nirmalas.ttf", font_size)
    draw.text ( (10,650), unicode_text, font=unicode_font, fill=font_color )

    img.save(directory+filename)

In [ ]:
# adding the Hindi text to images

d_directory = r"D:/subtitles/"

for i,caption in zip(all_files,df['Hindi']):  
    hindiText("D:/subtitlesHindi/",os.path.join(d_directory, i),str(caption),i)
    print(i,caption)

In [ ]:
# adding the English text to images

for i,caption in zip(all_files,df['English']):  
    addText("D:/subtitles/",os.path.join(d_directory, i),str(caption),i)
    print(i,caption)

In [ ]:
# border added to the images
from PIL import Image as pili, ImageOps as piliops

for filename in (os.listdir(r"D:/subtitlesHindi/")):
    if 'PNG' in filename:
#         print(filename)
        piliops.expand(pili.open(r"D:/subtitlesHindi/"+filename), border=(100,100),fill='black').save(r"D:/subtitlesHindi/"+filename)

In [ ]:
# Horizontally stacking the bordered images

import PIL
from PIL import Image as pili, ImageOps as piliops

def gridImage(imgList, d_directory):
    
    dim = imgList[0].split("SS")
    imgNumber = dim[1]
    imgs    = [ PIL.Image.open(i) for i in imgList ]
    # pick the image which is the smallest, and resize the others to match it (can be arbitrary image shape here)
    
    min_shape = sorted( [(np.sum(i.size), i.size ) for i in imgs])[0][1]
    print("Array size "+np.asarray( i.resize(min_shape) ) for i in imgs)
    imgs_comb = np.hstack( (np.asarray( i.resize(min_shape) ) for i in imgs ) )

    # save that beautiful picture
    imgs_comb = PIL.Image.fromarray(imgs_comb)
    imgs_comb.save( d_directory+imgNumber )

In [ ]:
import numpy as np
import PIL
from IPython.display import Image
from IPython.display import display
from PIL import Image as pili, ImageOps as piliops

# To get the random number of images per line (3 to 5) 

d_directory = "D:/subtitlesHindi/"
list_im = [ (os.path.join(d_directory, file)) for file in all_files if "PNG" in "D:/subtitlesHindi/"+file ]
while 0 < len(list_im):
    lines = 0
#     print("new page!")
    num_cols = old_num_cols = 0
    while lines < 6 and 0 < len(list_im):
        lines += 1
        while num_cols == old_num_cols:
            num_cols = np.random.randint(3, 6)
        old_num_cols = num_cols
        window = list_im[:num_cols]
#         print(window)
        gridImage(window, d_directory)
#         print((print (i) for i in window))
        list_im = list_im[num_cols:]


In [ ]:
# Vertically stacking the images 

def verticalImage(imgList, directory):
    
    imgNumber = imgList[0]
    imgs    = [ PIL.Image.open(directory+i) for i in imgList ]

    # pick the image which is the smallest, and resize the others to match it (can be arbitrary image shape here)
    min_shape = sorted( [(np.sum(i.size), i.size ) for i in imgs])[0][1]
    imgs_comb = np.vstack( (np.asarray( i.resize(min_shape) ) for i in imgs ) )

    # save that beautiful picture
    imgs_comb = PIL.Image.fromarray( imgs_comb)
    imgs_comb.save( directory+"dims"+imgNumber )

In [ ]:
# Sorting the horizontal stacked images in ascending order

horimages = []
for i in os.listdir(r"D:\vstackphotos"):
    horimages.append(i)
horimages = sort_nicely(horimages)


In [ ]:
# Horizontally stacked images now ready to be stacked vertically 

list1 = horimages[:5]
list2 = horimages[5:10]
list3 = horimages[10:15]

In [ ]:
# Vertically Stacking the Horizontal images

directory = r"D:/vstackphotos/"

verticalImage(list3, directory)

In [ ]:
# Converting images to pdf

from PIL import Image

image1 = Image.open(r'D:/vstackphotos/dims/dims1.PNG')
image2 = Image.open(r'D:/vstackphotos/dims/dims22.PNG')
image3 = Image.open(r'D:/vstackphotos/dims/dims43.PNG')

im1 = image1.convert('RGB')
im2 = image2.convert('RGB')
im3 = image3.convert('RGB')

imagelist = [im2,im3]
# for i in imagelist = ['D:/vstackphotos/dims1.PNG', 
#              'D:/vstackphotos/dims22.PNG', 
#              'D:/vstackphotos/dims43.PNG']
im1.save(r'D:/vstackphotos/dims/AvengersHindi.pdf',save_all=True, append_images=imagelist)